In [1]:
import pandas as pd

In [4]:
mm_df = pd.read_csv('Mass_Movement_Data/mmALL_073119_csv.csv')
mm_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16363 entries, 0 to 16362
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     16363 non-null  int64  
 1   country                16363 non-null  object 
 2   ccode                  16363 non-null  int64  
 3   year                   16363 non-null  int64  
 4   region                 16363 non-null  object 
 5   protest                16363 non-null  int64  
 6   protestnumber          16363 non-null  int64  
 7   startday               14514 non-null  float64
 8   startmonth             14514 non-null  float64
 9   startyear              14514 non-null  float64
 10  endday                 14514 non-null  float64
 11  endmonth               14514 non-null  float64
 12  endyear                14514 non-null  float64
 13  protesterviolence      15033 non-null  float64
 14  location               14493 non-null  object 
 15  pa

In [7]:
years_df = mm_df.loc[(mm_df['startyear'] > 2015) & (mm_df['startyear'] < 2019)]
#years_df.head()
years_df['region'].unique()

array(['North America', 'Central America', 'South America', 'Europe',
       'Africa', 'MENA', 'Asia', 'Oceania'], dtype=object)

In [19]:
regions_df = years_df.loc[(years_df['region'] == 'South America') | (years_df['region'] == 'Europe') | 
                          (years_df['region'] == 'Asia')]
regions_df.loc[regions_df['region'] == 'South America', 'country'].unique()
regions_df.loc[regions_df['region'] == 'Europe', 'country'].unique()
regions_df.loc[regions_df['region'] == 'Asia', 'country'].unique()
regions_df = regions_df.reset_index(drop = True)
regions_df.columns

Index(['id', 'country', 'ccode', 'year', 'region', 'protest', 'protestnumber',
       'startday', 'startmonth', 'startyear', 'endday', 'endmonth', 'endyear',
       'protesterviolence', 'location', 'participants_category',
       'participants', 'protesteridentity', 'protesterdemand1',
       'protesterdemand2', 'protesterdemand3', 'protesterdemand4',
       'stateresponse1', 'stateresponse2', 'stateresponse3', 'stateresponse4',
       'stateresponse5', 'stateresponse6', 'stateresponse7', 'sources',
       'notes'],
      dtype='object')

In [32]:
clean_columns_df = regions_df[['country', 'year', 'region', 'startday', 'startmonth', 'startyear', 
                               'endday', 'endmonth', 'endyear', 'location', 'participants_category', 
                               'participants', 'protesterdemand1', 'stateresponse1']]
#changing the data type
#start
clean_columns_df['startday'] = clean_columns_df['startday'].astype(int)
clean_columns_df['startmonth'] = clean_columns_df['startmonth'].astype(int)
clean_columns_df['startyear'] = clean_columns_df['startyear'].astype(int)
#end
clean_columns_df['endday'] = clean_columns_df['endday'].astype(int)
clean_columns_df['endmonth'] = clean_columns_df['endmonth'].astype(int)
clean_columns_df['endyear'] = clean_columns_df['endyear'].astype(int)

clean_columns_df['start'] = (clean_columns_df['startyear'].astype(str) + '-' + clean_columns_df['startmonth'].astype(str) + '-' + clean_columns_df['startday'].astype(str))
clean_columns_df['end'] = (clean_columns_df['endyear'].astype(str) + '-' + clean_columns_df['endmonth'].astype(str) + '-' + clean_columns_df['endday'].astype(str))
clean_columns_df.head()

clean_columns_df.columns

<ipython-input-32-8058182f70c0>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_columns_df['startday'] = clean_columns_df['startday'].astype(int)
<ipython-input-32-8058182f70c0>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_columns_df['startmonth'] = clean_columns_df['startmonth'].astype(int)
<ipython-input-32-8058182f70c0>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

Index(['country', 'year', 'region', 'startday', 'startmonth', 'startyear',
       'endday', 'endmonth', 'endyear', 'location', 'participants_category',
       'participants', 'protesterdemand1', 'stateresponse1', 'start', 'end'],
      dtype='object')

In [45]:
clean_df = clean_columns_df[['country', 'year', 'region', 'location', 'participants_category',
       'participants', 'protesterdemand1', 'stateresponse1', 'start', 'end']]
#for x in clean_df['start']:
    #print(x)

clean_df['start'] = [pd.Timestamp(x) for x in clean_df['start']]
clean_df['end'] = [pd.Timestamp(x) for x in clean_df['end']]

clean_df['protest_duration'] = clean_df['end'] - clean_df['start']
clean_df['protest_duration'].value_counts()
clean_df.to_csv('movement_data.csv')

<ipython-input-45-b927d554b35c>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['start'] = [pd.Timestamp(x) for x in clean_df['start']]
<ipython-input-45-b927d554b35c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['end'] = [pd.Timestamp(x) for x in clean_df['end']]
<ipython-input-45-b927d554b35c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

0 days      999
1 days       47
2 days       15
7 days       10
4 days        9
8 days        8
11 days       7
14 days       7
6 days        6
3 days        5
9 days        4
12 days       3
5 days        3
26 days       3
13 days       3
10 days       3
20 days       2
31 days       2
23 days       2
103 days      2
15 days       1
19 days       1
170 days      1
22 days       1
33 days       1
134 days      1
24 days       1
43 days       1
137 days      1
126 days      1
81 days       1
53 days       1
30 days       1
27 days       1
41 days       1
86 days       1
25 days       1
29 days       1
367 days      1
59 days       1
17 days       1
62 days       1
183 days      1
93 days       1
50 days       1
39 days       1
Name: protest_duration, dtype: int64